In [ ]:
import json, uuid, re, tempfile, os, copy, zipfile, shutil
from functools import reduce
from urllib.parse import urlparse
from arcgis import gis
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.features import FeatureSet
from arcgis.gis import GIS
import requests

import time

In [ ]:
source_gis = GIS()
print("Successfully logged in target as: " + source_gis.properties.user.username)

target_gis = GIS()
print("Successfully logged in source as: " + target_gis.properties.user.username)

In [ ]:
def create_empty_service(source_item,target_gis,is_view):
    create_parameters = {
        "name": source_item.name,
        "description": source_item.description,
        "capabilities": "Create,Query,Editing,Update,Delete,Sync",#need to turn off the editing, update, Delete as final publication function
        "spatialReference": {"wkid": 4326},
        "zDefault": 0,
        "properties": {
            "path": "",
            "description": "",
            "copyright": ""
        }
    }
    print(create_parameters)
    service_item = target_gis.content.create_service(name = source_item.name, 
                                                     service_type = 'featureService', 
                                                     create_params = create_parameters, 
                                                     tags = source_item.tags, 
                                                     snippet = source_item.snippet, 
                                                     item_properties = {'title':source_item.title,'typeKeywords':"source-"+source_item.id},
                                                     item_id = source_item.id,
                                                     is_view = is_view
                                                     )
    return service_item


In [ ]:
source_item = source_gis.content.get("d0d3163b23bc4128add151557eb22b77")
source_item_flc = FeatureLayerCollection.fromitem(source_item)
print(source_item.name)
name_available = target_gis.content.is_service_name_available(service_name = source_item.name, service_type = 'featureService')
print(name_available)
is_view = source_item_flc.properties['isView']
print(is_view)

In [ ]:
if name_available == True:
    service_item = create_empty_service(source_item,target_gis,is_view)
else:
    print('this service is existing')

In [ ]:
target_item = target_gis.content.get("d0d3163b23bc4128add151557eb22b77")
test_url = "https://ppgeospatial-dev.aurecongroup.digital/arcgis/rest/services/Hosted/service_f9397/FeatureServer/0"
#token = target_gis._con.token
#test_url = test_url+"?token="+token
trg_flc = FeatureLayerCollection.fromitem(target_item)

for layer_item in source_item.layers:

    src_url = layer_item.url
    src_fl_id = src_url.split("/")[-1]
    trg_fl_id = "/" + str(src_fl_id)
    print(src_url)
    json_str = json.loads(f'{layer_item.properties}')
    #print(json_str)

    del json_str['serviceItemId']
    if layer_item.properties['isView'] is True:
        update_str = {"url":test_url,"adminLayerInfo":{"viewLayerDefinition":{"sourceServiceName":"service_f9397","sourceLayerId":int(src_fl_id),"sourceLayerFields":"*"}}}
        print(update_str)
        json_str.update(update_str)
    #del json_str['sourceSpatialReference']
    temp_json = {"layers":[]}
    temp_json["layers"].append(json_str)

    #print(temp_json)
    trg_flc.manager.add_to_definition(temp_json)
    trg_flc.manager.refresh()

    if layer_item.properties['isView'] is True:

        trg_flc_fields = [field['name'] for field in trg_flc.layers[int(src_fl_id)].properties.fields]
        source_view_fields = [field['name'] for field in layer_item.properties.fields]
        print(trg_flc_fields)
        print(source_view_fields)
        update_str = {"fields":[]}
        for fields in trg_flc_fields:
            if fields in source_view_fields:
                json_str = {"name":fields,"visible":True}
                update_str["fields"].append(json_str)
            else:
                json_str = {"name":fields,"visible":False}
                update_str["fields"].append(json_str)

        #print(update_str)
        trg_flc.layers[int(src_fl_id)].manager.update_definition(update_str) 
print('all done')